In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
     |████████████████████████████████| 120 kB 75.7 MB/s 


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [33]:
train = pd.read_csv("/content/drive/MyDrive/Datasets/bnlp-resources/sentiment/consolidated/bn_all_train.tsv",encoding='utf-8', header=None, delimiter="\t")
test = pd.read_csv("/content/drive/MyDrive/Datasets/bnlp-resources/sentiment/consolidated/bn_all_test.tsv",encoding='utf-8', header=None, delimiter="\t")
dev = pd.read_csv("/content/drive/MyDrive/Datasets/bnlp-resources/sentiment/consolidated/bn_all_dev.tsv",encoding='utf-8', header=None, delimiter="\t")
train.columns = ['id', 'text', 'class_label']
test.columns = ['id', 'text', 'class_label']
dev.columns = ['id', 'text', 'class_label']
train.head()

,id,text,class_label
0,id,text,class_label
1,2760,আফতাব আপনি ভালো আছেন?,Neutral
2,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,Positive
3,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,Neutral
4,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,Negative


In [34]:
# del train['title']

In [35]:
dev.head()

,id,text,class_label
0,id,text,class_label
1,2,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,Positive
2,4,সৌম্যকে বাদ দেওয়া হোক,Negative
3,5,"প্রথমটি হচ্ছে, কোচ অত:পর সাকিব,সাকিব আর সাকিবর...",Positive
4,20,আমরা বাঙালিরা অলপতে খুশি।একবার ভাল খেললে দেমাগ...,Negative


In [36]:
train.drop(0, inplace=True, axis=0)
test.drop(0, inplace=True, axis=0)
dev.drop(0, inplace=True, axis=0)
labels = list(set(train['class_label']))
print(labels)

['Negative', 'Positive', 'Neutral', 'positive', 'negative']


In [37]:
len(labels)

5

In [38]:
train['class_label'] = train['class_label'].replace({'Negative':0, 'Positive':1, 'Neutral':2, 'positive':1, 'negative':0})
test['class_label'] = test['class_label'].replace({'Negative':0, 'Positive':1, 'Neutral':2, 'positive':1, 'negative':0})
dev['class_label'] = dev['class_label'].replace({'Negative':0, 'Positive':1, 'Neutral':2, 'positive':1, 'negative':0})

In [39]:
# train['class_label'] = train['class_label'].replace({'Anger/Disgust': 0, 'None': 1, 'Joy': 2, 'Sadness': 3, 'Fear/Surprise': 4})
# test['class_label'] = test['class_label'].replace({'Anger/Disgust': 0, 'None': 1, 'Joy': 2, 'Sadness': 3, 'Fear/Surprise': 4})
# dev['class_label'] = dev['class_label'].replace({'Anger/Disgust': 0, 'None': 1, 'Joy': 2, 'Sadness': 3, 'Fear/Surprise': 4})

In [40]:
labels = list(set(train['class_label']))
print(labels)
del train['id']
train.head()

[0, 1, 2]


,text,class_label
1,আফতাব আপনি ভালো আছেন?,2
2,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,1
3,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,2
4,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,0
5,শুভকামনা রাজ্জাক ভাই।,0


In [41]:
del test['id']
test.head()

,text,class_label
1,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,0
2,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",0
3,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,0
4,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,0
5,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,0


In [42]:
del dev['id']
dev.head()

,text,class_label
1,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,1
2,সৌম্যকে বাদ দেওয়া হোক,0
3,"প্রথমটি হচ্ছে, কোচ অত:পর সাকিব,সাকিব আর সাকিবর...",1
4,আমরা বাঙালিরা অলপতে খুশি।একবার ভাল খেললে দেমাগ...,0
5,"মুমিনুলের পর, রাজের রাজসিক প্রত্যাবর্তন।",1


In [43]:
train_texts = train['text'].values.tolist()
train_labels = train['class_label'].values.tolist()
test_texts = test['text'].values.tolist()
dev_texts = dev['text'].values.tolist()
dev_labels = dev['class_label'].values.tolist()

In [44]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import Trainer,TrainingArguments

In [45]:
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/CSE498',num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/pet_bnbert/p0-i0',num_labels=3)
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/pruned_bnbert',num_labels=3)

Didn't find file /content/drive/MyDrive/Models/pruned_bnbert/tokenizer.json. We won't load it.
Didn't find file /content/drive/MyDrive/Models/pruned_bnbert/added_tokens.json. We won't load it.
loading file /content/drive/MyDrive/Models/pruned_bnbert/vocab.txt
loading file None
loading file None
loading file /content/drive/MyDrive/Models/pruned_bnbert/special_tokens_map.json
loading file /content/drive/MyDrive/Models/pruned_bnbert/tokenizer_config.json


In [46]:
train_encodings = tokenizer(train_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')
test_encodings = tokenizer(test_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')
dev_encodings = tokenizer(dev_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')

In [47]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
## Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

In [48]:
train_dataset = SentimentDataset(train_encodings, train_labels)
dev_dataset = SentimentDataset(dev_encodings, dev_labels)
test_dataset = SentimentTestDataset(test_encodings)

In [49]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    y_score = pred
    y_true = labels

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true, y_score, average='weighted')
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1, "recall": recall, "precision": precision}

In [50]:
training_args = TrainingArguments(
    output_dir='/content/result',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True,
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/CSE498',num_labels=3)
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/pet_bnbert/p0-i0', num_labels=3, ignore_mismatched_sizes=True)
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/pruned_bnbert', num_labels=3)

# for name, param in model.named_parameters():
#      if name.startswith("bert.encoder.layer.1"): # choose whatever you like here
#         param.requires_grad = False

trainer = Trainer(
    model=model,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=dev_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)


loading configuration file /content/drive/MyDrive/Models/pruned_bnbert/config.json
You passed along `num_labels=2` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2'}. The number of labels wil be overwritten to 3.
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Models/pruned_bnbert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 101,
  "classifier_dropout": null,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 102,
  "eos_token_ids": null,
  "finetuning_task": "asl",
  "gap_size": 0,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 1248,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2

In [52]:
trainer.train()

***** Running training *****
  Num examples = 19044
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 17880


Step,Training Loss,Validation Loss,Accuracy,F1 Score,Recall,Precision
500,0.840800,0.623313,0.755430,0.753972,0.755430,0.755328
1000,0.575800,0.572868,0.780995,0.780365,0.780995,0.781183
1500,0.422100,0.586711,0.793439,0.794444,0.793439,0.798071
2000,0.257000,0.656750,0.787557,0.785911,0.787557,0.785561
2500,0.127300,0.829543,0.792760,0.793554,0.792760,0.796999
3000,0.073700,0.896616,0.796380,0.796400,0.796380,0.796575
3500,0.035300,1.101858,0.794796,0.795289,0.794796,0.801258
4000,0.023400,1.154033,0.796833,0.798084,0.796833,0.800595
4500,0.014500,1.307839,0.789819,0.783863,0.789819,0.787761
5000,0.011400,1.322790,0.795928,0.794650,0.795928,0.794072


***** Running Evaluation *****
  Num examples = 4420
  Batch size = 64
Saving model checkpoint to /content/result/checkpoint-500
Configuration saved in /content/result/checkpoint-500/config.json
Model weights saved in /content/result/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4420
  Batch size = 64
Saving model checkpoint to /content/result/checkpoint-1000
Configuration saved in /content/result/checkpoint-1000/config.json
Model weights saved in /content/result/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4420
  Batch size = 64
Saving model checkpoint to /content/result/checkpoint-1500
Configuration saved in /content/result/checkpoint-1500/config.json
Model weights saved in /content/result/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4420
  Batch size = 64
Saving model checkpoint to /content/result/checkpoint-2000
Configuration saved in /content/result/checkpoint-2000/config.j

TrainOutput(global_step=17880, training_loss=0.06925932878849224, metrics={'train_runtime': 6039.7461, 'train_samples_per_second': 94.593, 'train_steps_per_second': 2.96, 'total_flos': 8403316072243200.0, 'train_loss': 0.06925932878849224, 'epoch': 30.0})

In [53]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4420
  Batch size = 64


{'eval_loss': 0.5728675723075867,
 'eval_accuracy': 0.7809954751131222,
 'eval_f1_score': 0.7803651045799168,
 'eval_recall': 0.7809954751131222,
 'eval_precision': 0.7811828632030733,
 'eval_runtime': 12.8901,
 'eval_samples_per_second': 342.899,
 'eval_steps_per_second': 5.431,
 'epoch': 30.0}

In [54]:
import time
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = elapsed_time / 60
  elapsed_secs = elapsed_time - (elapsed_mins * 60)
  return elapsed_mins, elapsed_secs

In [55]:
test['class_label'] = 0
test_texts = test['text'].values.tolist()
test_labels = test["class_label"].values.tolist() 
test_encodings = tokenizer(test_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')
test_dataset = SentimentDataset(test_encodings, test_labels)

start_time = time.time()
preds = trainer.predict(test_dataset=test_dataset)
end_time = time.time()

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()# convert tensors to numpy array

newdf = pd.DataFrame(predictions,columns=['Negative_0', 'Neutral_1', 'Positive_2'])
newdf.head()
def labels(x):
  if x == 0:
    return 'Negative_0'
  elif x == 1:
    return 'Neutral_1'
  else:
    return 'Positive_2'

results = np.argmax(predictions,axis=1)
test['class_label'] = results
test['class_label'] = test['class_label'].map(labels)
test.head()


***** Running Prediction *****
  Num examples = 4402
  Batch size = 64


,text,class_label
1,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,Negative_0
2,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",Negative_0
3,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,Negative_0
4,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,Negative_0
5,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,Negative_0


In [56]:
epoch_mins, epoch_secs = epoch_time(start_time, end_time)

In [57]:
print(epoch_mins, epoch_secs)

0.2538611928621928 0.0


## **Model_convert_flax(pt)_to_(.bin)**

In [ ]:
# from transformers import BertModel

In [ ]:
# model = BertModel.from_pretrained(
#     "/content/drive/MyDrive/Models/Pruned_model/model_1", from_flax=True
# )